# 🤗 Welcome to AdalFlow!
## The PyTorch library to auto-optimize any LLM task pipelines

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1v_AL45U6r9Bp4P4Ykcuus5YXxevB1Ai8?usp=drive_link)

Thanks for trying us out, we're here to provide you with the best LLM application development experience you can dream of 😊 any questions or concerns you may have, [come talk to us on discord,](https://discord.gg/ezzszrRZvT) we're always here to help! ⭐ <i>Star us on <a href="https://github.com/SylphAI-Inc/AdalFlow">Github</a> </i> ⭐


# Quick Links

Github repo: https://github.com/SylphAI-Inc/AdalFlow

Full Tutorials: https://adalflow.sylph.ai/index.html#.

Deep dive on each API: check out the [developer notes](https://adalflow.sylph.ai/tutorials/index.html).

Common use cases along with the auto-optimization:  check out [Use cases](https://adalflow.sylph.ai/use_cases/index.html).

# Outline

This is a quick introduction of what AdalFlow is capable of. We will cover:

* Simple Chatbot with structured output
* RAG task pipeline + Data processing pipeline
* Agent

**Next: Try our [auto-optimization](https://colab.research.google.com/drive/1n3mHUWekTEYHiBdYBTw43TKlPN41A9za?usp=sharing)**


# Installation

1. Use `pip` to install the `adalflow` Python package. We will need `openai`, `groq`, and `faiss`(cpu version) from the extra packages.

  ```bash
  pip install adalflow[openai,groq,faiss-cpu]
  ```
2. Setup  `openai` and `groq` API key in the environment variables

In [ ]:
from IPython.display import clear_output

!pip install -U adalflow[openai,groq,faiss-cpu]

clear_output()

## Set Environment Variables

Run the following code and pass your api key.

Note: for normal `.py` projects, follow our [official installation guide](https://lightrag.sylph.ai/get_started/installation.html).

*Go to [OpenAI](https://platform.openai.com/docs/introduction) and [Groq](https://console.groq.com/docs/) to get API keys if you don't already have.*

In [ ]:
import os

from getpass import getpass

# Prompt user to enter their API keys securely
openai_api_key = getpass("Please enter your OpenAI API key: ")
groq_api_key = getpass("Please enter your GROQ API key: ")


# Set environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['GROQ_API_KEY'] = groq_api_key

print("API keys have been set.")

Please enter your OpenAI API key: ··········
Please enter your GROQ API key: ··········
API keys have been set.




# 😇 First ChatBot

We will start with a single turn chatbot which will explain concepts with ``explanation`` and ``example``. To achieve this, we will build a simple pipeline to get the **structured output** as ``QAOutput``.


##Well-designed Base Classes


We will use this use case to demonstrate how to leverage our two and only powerful base classes: `Component` as building blocks for the pipeline and `DataClass` to ease the data interaction with LLMs.




In [ ]:
# Prepare data and template [jinja2 syntax]

from dataclasses import dataclass, field
from typing import Dict

import adalflow as adal
from adalflow.components.model_client import GroqAPIClient

@dataclass
class QAOutput(adal.DataClass):
    explanation: str = field(
        metadata={"desc": "A brief explanation of the concept in one sentence."}
    )
    example: str = field(metadata={"desc": "An example of the concept in a sentence."})
    __output_fields__ = ["explanation", "example"] # this automatically controls output fields in the same order you provided



qa_template = r"""<SYS>
You are a helpful assistant.
<OUTPUT_FORMAT>
{{output_format_str}}
</OUTPUT_FORMAT>
</SYS>
User: {{input_str}}"""

In [ ]:
# Create the task pipeline

class QA(adal.Component):
    def __init__(self, model_client: adal.ModelClient, model_kwargs: Dict):
        super().__init__()

        parser = adal.DataClassParser(data_class=QAOutput, return_data_class=True)
        self.generator = adal.Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
            prompt_kwargs={"output_format_str": parser.get_output_format_str()},
            output_processors=parser,
        )

    def call(self, query: str):
        return self.generator.call({"input_str": query})

    async def acall(self, query: str):
        return await self.generator.acall({"input_str": query})

## Clear Pipeline Structure

Simply by using `print(qa)`, you can see the pipeline structure, which helps users understand any LLM workflow quickly, especially when the pipeline is complicated.



In [ ]:
# Instantiate the QA class

qa = QA(
    model_client=GroqAPIClient(),
    model_kwargs={"model": "llama3-8b-8192"},
)

print(qa)

output_format_str: Your output should be formatted as a standard JSON instance with the following schema:
```
{
    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",
    "example": "An example of the concept in a sentence. (str) (required)"
}
```
-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!
-Use double quotes for the keys and string values.
-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.
-Follow the JSON formatting conventions.
cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-8b-8192.db
QA(
  (generator): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, 
    (prompt): Prompt(
      template: <SYS>
      You are a helpful assistant.
      <OUTPUT_FORMAT>
      {{output_format_str}}
      </OUTPUT_FORMAT>
      </SYS>
      User: {{input_str}}, prompt_kwargs: {'output_format_str': 'Your output should be 

In [ ]:
# call the qa and check the output

qa("What is LLM?")

GeneratorOutput(id=None, data=QAOutput(explanation='LLM stands for Large Language Model, a type of artificial intelligence designed to process and generate human-like language.', example='LLMs are often used in applications such as language translation, text generation, and chatbots.'), error=None, usage=CompletionUsage(completion_tokens=58, prompt_tokens=170, total_tokens=228), raw_response='```\n{\n    "explanation": "LLM stands for Large Language Model, a type of artificial intelligence designed to process and generate human-like language.",\n    "example": "LLMs are often used in applications such as language translation, text generation, and chatbots."\n}\n```', metadata=None)

In [ ]:
# display the prompt only

qa.generator.print_prompt(
        output_format_str=qa.generator.output_processors.get_output_format_str(),
        input_str="What is LLM?",
)

output_format_str: Your output should be formatted as a standard JSON instance with the following schema:
```
{
    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",
    "example": "An example of the concept in a sentence. (str) (required)"
}
```
-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!
-Use double quotes for the keys and string values.
-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.
-Follow the JSON formatting conventions.
Prompt:
______________________
<SYS>
You are a helpful assistant.
<OUTPUT_FORMAT>
Your output should be formatted as a standard JSON instance with the following schema:
```
{
    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",
    "example": "An example of the concept in a sentence. (str) (required)"
}
```
-Make sure to always enclose the JSON output in 

'<SYS>\nYou are a helpful assistant.\n<OUTPUT_FORMAT>\nYour output should be formatted as a standard JSON instance with the following schema:\n```\n{\n    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",\n    "example": "An example of the concept in a sentence. (str) (required)"\n}\n```\n-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!\n-Use double quotes for the keys and string values.\n-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.\n-Follow the JSON formatting conventions.\n</OUTPUT_FORMAT>\n</SYS>\nUser: What is LLM?'

## Model-Agnostic

You can switch to any model simply by using a different model_client (provider) and model_kwargs.
Let's use OpenAI's gpt-3.5-turbo model on the same pipeline.

In [ ]:
from adalflow.components.model_client import OpenAIClient

qa_with_gpt = QA(
    model_client=OpenAIClient(),
    model_kwargs={"model": "gpt-3.5-turbo"}
)

qa_with_gpt("What is LLM?")

output_format_str: Your output should be formatted as a standard JSON instance with the following schema:
```
{
    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",
    "example": "An example of the concept in a sentence. (str) (required)"
}
```
-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!
-Use double quotes for the keys and string values.
-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.
-Follow the JSON formatting conventions.
cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db


GeneratorOutput(id=None, data=QAOutput(explanation='LLM stands for Large Language Model, which is a type of artificial intelligence model that can process and generate human language.', example='GPT-3 is an example of an LLM that is capable of generating human-like text in a variety of contexts.'), error=None, usage=CompletionUsage(completion_tokens=66, prompt_tokens=167, total_tokens=233), raw_response='```json\n{\n    "explanation": "LLM stands for Large Language Model, which is a type of artificial intelligence model that can process and generate human language.",\n    "example": "GPT-3 is an example of an LLM that is capable of generating human-like text in a variety of contexts."\n}\n```', metadata=None)

# 🤗 First RAG

Different from other libraries, the RAG pipeline consists of (1) a task pipeline consists of a retriever and a generator (2) a data pipeline that works with local/cloud db to preprocess and persist data.

This suits the real product environment. And if the data is embedded and used only once-off on the fly, you can always skip the data storage.

## Use Config

We will put all configurations together in `config` as dict.

In [ ]:
configs = {
    "embedder": {
        "batch_size": 100,
        "model_kwargs": {
            "model": "text-embedding-3-small",
            "dimensions": 256,
            "encoding_format": "float",
        },
    },
    "retriever": {
        "top_k": 2,
    },
    "generator": {
        "model": "gpt-3.5-turbo",
        "temperature": 0.3,
        "stream": False,
    },
    "text_splitter": {
        "split_by": "word",
        "chunk_size": 400,
        "chunk_overlap": 200,
    },
}

## Prepare data pipeline

We will use local data base `LocalDB` and `core.data_process` to create a data processing pipeline. This data pipeline will split documents into chunks and work with `LocalDB` to persis the transformed/processed documents in local file `index.faiss` (pickle format).

Data pipeline requires a sequence of `Document` as inputs.

In [ ]:
from adalflow.components.data_process import (
    RetrieverOutputToContextStr,
    ToEmbeddings,
    TextSplitter,
)

from adalflow.core.types import Document, ModelClientType


def prepare_data_pipeline():
    splitter = TextSplitter(**configs["text_splitter"])
    embedder = adal.Embedder(
        model_client=ModelClientType.OPENAI(),
        model_kwargs=configs["embedder"]["model_kwargs"],
    )
    embedder_transformer = ToEmbeddings(
        embedder=embedder, batch_size=configs["embedder"]["batch_size"]
    )
    data_transformer = adal.Sequential(splitter, embedder_transformer) # sequential will chain together splitter and embedder
    return data_transformer

In [ ]:
data_transformer = prepare_data_pipeline()
data_transformer

Sequential(
  (0): TextSplitter(split_by=word, chunk_size=400, chunk_overlap=200)
  (1): ToEmbeddings(
    batch_size=100
    (embedder): Embedder(
      model_kwargs={'model': 'text-embedding-3-small', 'dimensions': 256, 'encoding_format': 'float'}, 
      (model_client): OpenAIClient()
    )
    (batch_embedder): BatchEmbedder(
      (embedder): Embedder(
        model_kwargs={'model': 'text-embedding-3-small', 'dimensions': 256, 'encoding_format': 'float'}, 
        (model_client): OpenAIClient()
      )
    )
  )
)

In [ ]:
# Prepare documents for data transformer

doc1 = Document(
        meta_data={"title": "Li Yin's profile"},
        text="My name is Li Yin, I love rock climbing" + "lots of nonsense text" * 500,
        id="doc1",
)
doc2 = Document(
    meta_data={"title": "Interviewing Li Yin"},
    text="lots of more nonsense text" * 250
    + "Li Yin is an AI researcher and a software engineer"
    + "lots of more nonsense text" * 250,
    id="doc2",
)

doc1

Document(id=doc1, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector=[], parent_doc_id=None, order=None, score=None)

In [ ]:
# transform the data

transformed_documents = data_transformer([doc1, doc2])

Batch embedding documents: 100%|██████████| 1/1 [00:00<00:00,  2.06it/s]
Adding embeddings to documents from batch: 1it [00:00, 7002.18it/s]


## Transformed documents

From the following visualization, we will see `doc1` is splitted into 7 chunks and `doc2` is splitted into 10 chunks. We get this relation from reading the `transformed_documents`, the `parent_doc_id` field.

Note: For `text` and `vector`, we dont show the full text or the full vector as it is rather long. You can access each field directly to visualize the full value

In [ ]:
# visualize the transformed data
transformed_documents

[Document(id=38cf2f19-ceab-46f7-9702-0aa2fa80b06f, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None),
 Document(id=b0f13ef5-a6d9-42f7-865f-f3f9f9bfa2b1, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=1, score=None),
 Document(id=efe655dd-703b-4db6-bd99-1d13c6d43243, text='nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlot...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=2, score=None),
 Document(id=b52d4e0e-686a-43f3-b5a2-1361231e17ec, text='of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense text...', meta_data={'title': "Li Yin's profile"}, v

## Use LocalDB

We will use localdb to manage the `documents`, `transformers`, and the persistance of the transformed documents. This resembles more of the production environment where the embeddings and documents are often handled in data base and can be reused to save cost.

In [ ]:
from typing import List
from adalflow.core.db import LocalDB


def prepare_database_with_index(docs: List[Document], index_path: str = "index.faiss"):
    if os.path.exists(index_path):
        return None
    db = LocalDB()
    db.load(docs)
    data_transformer = prepare_data_pipeline()
    db.transform(data_transformer, key="data_transformer")
    # store
    db.save_state(index_path)
    print(db)

In [ ]:
# prepare the database for retriever

prepare_database_with_index([doc1, doc2], index_path="index.faiss")

Batch embedding documents: 100%|██████████| 1/1 [00:00<00:00,  1.86it/s]
Adding embeddings to documents from batch: 1it [00:00, 8065.97it/s]

LocalDB(name='LocalDB', items=[Document(id=doc1, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=doc2, text='lots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonsense ...', meta_data={'title': 'Interviewing Li Yin'}, vector=[], parent_doc_id=None, order=None, score=None)], transformed_items={'data_transformer': [Document(id=7a526a52-76f0-4bd5-bd41-d02facb282af, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=40ce0741-d9f2-4243-a26b-32e1fddd8f71, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's profile"}, vect

## Load from file

LocalDB `save_state` not only persist the transformed documents, but also the `data_transformer`.

This is really helpful as your retriever needs to have a matching `embedder` to embed the string query. Saving the transformer lets you verify and know what embedder you need to pass to Retriever.

In [ ]:
# test the database loading

db = LocalDB.load_state("index.faiss")
db

LocalDB(name='LocalDB', items=[Document(id=doc1, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=doc2, text='lots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonsense ...', meta_data={'title': 'Interviewing Li Yin'}, vector=[], parent_doc_id=None, order=None, score=None)], transformed_items={'data_transformer': [Document(id=7a526a52-76f0-4bd5-bd41-d02facb282af, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=40ce0741-d9f2-4243-a26b-32e1fddd8f71, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's profile"}, vect

In [ ]:
# test data fetching from local db

db.get_transformed_data("data_transformer")

[Document(id=4c1fa2e1-581d-4b57-bbb6-c8f5d5d4da4d, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None),
 Document(id=55ea86ea-175b-4a6b-ac4e-df303cb8440f, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=1, score=None),
 Document(id=464775cb-637a-43bc-be85-908a7a35e9c4, text='nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlot...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=2, score=None),
 Document(id=0447254c-a093-4e56-9ad1-a7a8730742d0, text='of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense text...', meta_data={'title': "Li Yin's profile"}, v

## RAG pipeline

Now, we will create a RAG pipeline, it consists of:
* db (we will load from index_path), we will use `data_transformer` as the key to load the transformed documents.
* `FAISSRetriever` which will use embeddings to perform semantic search, and return similarity score in range [0, 1].
* `RetrieverOutputToContextStr`: this will convert the retrieved documents to a single str.
* `Generator`: we will use a simple `JsonParser` to output a dict with field `answer`.

In [ ]:
from typing import Optional, Any

from adalflow.core.string_parser import JsonParser
from adalflow.components.retriever.faiss_retriever import FAISSRetriever


rag_prompt_task_desc = r"""
You are a helpful assistant.

Your task is to answer the query that may or may not come with context information.
When context is provided, you should stick to the context and less on your prior knowledge to answer the query.

Output JSON format:
{
    "answer": "The answer to the query",
}"""


class RAG(adal.Component):

    def __init__(self, index_path: str = "index.faiss"):
        super().__init__()

        self.db = LocalDB.load_state(index_path)

        self.transformed_docs: List[Document] = self.db.get_transformed_data(
            "data_transformer"
        )
        embedder = adal.Embedder(
            model_client=ModelClientType.OPENAI(),
            model_kwargs=configs["embedder"]["model_kwargs"],
        )
        # map the documents to embeddings
        self.retriever = FAISSRetriever(
            **configs["retriever"],
            embedder=embedder,
            documents=self.transformed_docs,
            document_map_func=lambda doc: doc.vector,
        )
        self.retriever_output_processors = RetrieverOutputToContextStr(deduplicate=True)

        self.generator = adal.Generator(
            prompt_kwargs={
                "task_desc_str": rag_prompt_task_desc,
            },
            model_client=OpenAIClient(),
            model_kwargs=configs["generator"],
            output_processors=JsonParser(),
        )

    def generate(self, query: str, context: Optional[str] = None) -> Any:
        if not self.generator:
            raise ValueError("Generator is not set")

        prompt_kwargs = {
            "context_str": context,
            "input_str": query,
        }
        response = self.generator(prompt_kwargs=prompt_kwargs)
        return response

    def call(self, query: str) -> Any:
        retrieved_documents = self.retriever(query)
        # fill in the document
        for i, retriever_output in enumerate(retrieved_documents):
            retrieved_documents[i].documents = [
                self.transformed_docs[doc_index]
                for doc_index in retriever_output.doc_indices
            ]

        print(f"retrieved_documents: \n {retrieved_documents}\n")
        context_str = self.retriever_output_processors(retrieved_documents)

        print(f"context_str: \n {context_str}\n")

        return self.generate(query, context=context_str), retrieved_documents

In [ ]:
# initialize rag and visualize its structure

rag = RAG(index_path="index.faiss")
rag

cache_path: /root/.adalflow/cache_OpenAIClient_gpt-3.5-turbo.db


RAG(
  (db): LocalDB(name='LocalDB', items=[Document(id=doc1, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector=[], parent_doc_id=None, order=None, score=None), Document(id=doc2, text='lots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonsense ...', meta_data={'title': 'Interviewing Li Yin'}, vector=[], parent_doc_id=None, order=None, score=None)], transformed_items={'data_transformer': [Document(id=7a526a52-76f0-4bd5-bd41-d02facb282af, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=40ce0741-d9f2-4243-a26b-32e1fddd8f71, text='textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nons...', meta_data={'title': "Li Yin's pr

In [ ]:
# run RAG end to end

query = "What is Li Yin's hobby and profession?"

response, retrieved_documents = rag.call(query)
print(response)

calling the call method
retrieved_documents: 
 [RetrieverOutput(doc_indices=[0, 11], doc_scores=[0.7120000123977661, 0.6650000214576721], query="What is Li Yin's hobby and profession?", documents=[Document(id=7a526a52-76f0-4bd5-bd41-d02facb282af, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=b05e9391-0943-46e2-b107-e25150325fbe, text='textlots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonse...', meta_data={'title': 'Interviewing Li Yin'}, vector='len: 256', parent_doc_id=doc2, order=4, score=None)])]

context_str: 
  My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots

In [ ]:
# get the documents from retriever

print(retrieved_documents[0].documents)

text1, text2= retrieved_documents[0].documents[0].text, retrieved_documents[0].documents[1].text

print("rock climbing" in text1, text1)
print("software engineer" in text2, text2)


# try to manually search software engineer in the printout, you will find the key word that match the retrieval

[Document(id=7a526a52-76f0-4bd5-bd41-d02facb282af, text='My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense te...', meta_data={'title': "Li Yin's profile"}, vector='len: 256', parent_doc_id=doc1, order=0, score=None), Document(id=b05e9391-0943-46e2-b107-e25150325fbe, text='textlots of more nonsense textlots of more nonsense textlots of more nonsense textlots of more nonse...', meta_data={'title': 'Interviewing Li Yin'}, vector='len: 256', parent_doc_id=doc2, order=4, score=None)]
True My name is Li Yin, I love rock climbinglots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots of nonsense textlots

# 🤚 First Agent

We will try React agent which calls tools sequentially. Please refer to our tutorials [tools](https://adalflow.sylph.ai/tutorials/tool_helper.html) and [agent](https://adalflow.sylph.ai/tutorials/agent.html) for more details.

In [ ]:

from adalflow.components.agent import ReActAgent
from adalflow.core import ModelClientType


def multiply(a: int, b: int) -> int:
   """
   Multiply two numbers.
   """
   return a * b

async def add(a: int, b: int) -> int:
   """
   Add two numbers.
   """
   return a + b

def divide(a: float, b: float) -> float:
   """
   Divide two numbers.
   """
   return float(a) / b

llama3_model_kwargs = {
   "model": "llama3-70b-8192",  # llama3 70b works better than 8b here.
   "temperature": 0.0,
}
gpt_model_kwargs = {
   "model": "gpt-3.5-turbo",
   "temperature": 0.0,
}

We will compare our React agent's response with the vanilla LLM.

In [ ]:
def test_react_agent(model_client: adal.ModelClient, model_kwargs: dict):
   tools = [multiply, add, divide]
   queries = [
      "What is the capital of France? and what is 465 times 321 then add 95297 and then divide by 13.2?",
      "Give me 5 words rhyming with cool, and make a 4-sentence poem using them",
   ]

   # vanilla LLM
   generator = adal.Generator(
      model_client=model_client,
      model_kwargs=model_kwargs,
   )

   # agent
   react = ReActAgent(
      max_steps=6,
      add_llm_as_fallback=True,
      tools=tools,
      model_client=model_client,
      model_kwargs=model_kwargs,
   )
   # print(react)

   for query in queries:
      print(f"Query: {query}")
      agent_response = react.call(query)
      llm_response = generator.call(prompt_kwargs={"input_str": query})
      print(f"Agent response: {agent_response}")
      print(f"LLM response: {llm_response}")
      print("")

In [ ]:
test_react_agent(ModelClientType.GROQ(), llama3_model_kwargs)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-70b-8192.db
cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-70b-8192.db
cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-70b-8192.db
Query: What is the capital of France? and what is 465 times 321 then add 95297 and then divide by 13.2?
2024-08-23 19:27:35 - [react.py:284:call] - input_query: What is the capital of France? and what is 465 times 321 then add 95297 and then divide by 13.2?
2024-08-23 19:27:36 - [react.py:264:_run_one_step] - Step 1: 
StepOutput(step=1, action=FunctionExpression(thought="Let's break down the query into subqueries and start with the first one.", action='llm_tool(input="What is the capital of France?")'), function=Function(thought=None, name='llm_tool', args=[], kwargs={'input': 'What is the capital of France?'}), observation='The capital of France is Paris.')
_______

2024-08-23 19:27:36 - [react.py:264:_run_one_step] - Step 2: 
StepOutput(step=2, action=FunctionExpression(thought="Now, le

ReActAgent itself is a task pipeline that consists of Generator and output paraser and a for loop to do multiple steps.

In [ ]:
# lets visualize react structure

tools = [multiply]
react = ReActAgent(
      max_steps=6,
      add_llm_as_fallback=True,
      tools=tools,
      model_client=ModelClientType.GROQ(),
      model_kwargs=llama3_model_kwargs,
   )

print(react)

cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-70b-8192.db
cache_path: /root/.adalflow/cache_GroqAPIClient_llama3-70b-8192.db
ReActAgent(
  max_steps=6, add_llm_as_fallback=True, 
  (tool_manager): ToolManager(Tools: [FunctionTool(fn: <function multiply at 0x7b75c25541f0>, async: False, definition: FunctionDefinition(func_name='multiply', func_desc='multiply(a: int, b: int) -> int\n\n   Multiply two numbers.\n   ', func_parameters={'type': 'object', 'properties': {'a': {'type': 'int'}, 'b': {'type': 'int'}}, 'required': ['a', 'b']})), FunctionTool(fn: <function ReActAgent._init_tools.<locals>.llm_tool at 0x7b75bc9ce830>, async: False, definition: FunctionDefinition(func_name='llm_tool', func_desc="llm_tool(input: str) -> str\nI answer any input query with llm's world knowledge. Use me as a fallback tool or when the query is simple.", func_parameters={'type': 'object', 'properties': {'input': {'type': 'str'}}, 'required': ['input']})), FunctionTool(fn: <function ReActAgent._init

# Next: Try our [auto-optimization](https://colab.research.google.com/drive/1n3mHUWekTEYHiBdYBTw43TKlPN41A9za?usp=sharing)

# Issues and feedback

If you encounter any issues, please report them here: [GitHub Issues](https://github.com/SylphAI-Inc/LightRAG/issues).

For feedback, you can use either the [GitHub discussions](https://github.com/SylphAI-Inc/LightRAG/discussions) or [Discord](https://discord.gg/ezzszrRZvT).